In [ ]:
import pandas as pd
import requests
from lxml import html
import concurrent.futures
import re
import os
import time
from tqdm import tqdm
import logging

# Nastavení loggeru
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Funkce pro scrapování jedné URL adresy
def scrape_url(url):
    try:
        # Přidání timeout pro zabránění zablokování
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code != 200:
            return url, None, None
        
        # Parsování HTML
        tree = html.fromstring(response.content)
        
        # Extrakce dat podle prvního XPath
        xpath_result_1 = tree.xpath("//span[@itemprop='itemListElement']/a/span[@itemprop='name']/text()")
        result_1 = xpath_result_1[0] if xpath_result_1 else None
        
        # Extrakce dat podle druhého XPath (regex)
        page_type = None
        match = re.search(r'"pageType"\s*:\s*"(.*?)"', response.text)
        if match:
            page_type = match.group(1)
        
        return url, result_1, page_type
    
    except Exception as e:
        logger.error(f"Chyba při scrapování {url}: {str(e)}")
        return url, None, None

# Hlavní funkce
def main():
    start_time = time.time()
    
    # Vytvoření složky pro výstup, pokud neexistuje
    if not os.path.exists("scraped"):
        os.makedirs("scraped")
    
    # Načtení CSV souboru
    logger.info("Načítání CSV souboru...")
    df = pd.read_csv("50-100_url.csv")
    
    # Kontrola, zda soubor obsahuje sloupec s URL adresami
    if 'url' not in df.columns:
        # Pokud sloupec nemá název 'url', předpokládáme, že první sloupec obsahuje URL adresy
        df.columns = ['url'] + list(df.columns)[1:]
    
    # Získání seznamu URL adres
    urls = df['url'].tolist()
    
    # Inicializace seznamů pro výsledky
    results = []
    
    # Paralelní zpracování URL adres
    logger.info("Začínám scrapování...")
    
    # Určení optimálního počtu workerů (obvykle 2-4x počet CPU jader)
    max_workers = min(32, os.cpu_count() * 4)
    
    # Zpracování po dávkách pro lepší správu paměti
    batch_size = 1000
    total_batches = (len(urls) + batch_size - 1) // batch_size
    
    for i in range(0, len(urls), batch_size):
        batch_urls = urls[i:i+batch_size]
        batch_results = []
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Použití tqdm pro zobrazení průběhu
            futures = {executor.submit(scrape_url, url): url for url in batch_urls}
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(batch_urls), desc=f"Dávka {i//batch_size + 1}/{total_batches}"):
                batch_results.append(future.result())
        
        # Přidání výsledků dávky do celkových výsledků
        results.extend(batch_results)
        
        # Průběžné ukládání výsledků (pro případ selhání)
        if (i + batch_size) % (batch_size * 10) == 0 or (i + batch_size) >= len(urls):
            temp_df = pd.DataFrame(results, columns=['url', 'xpath_result', 'page_type'])
            temp_df.to_csv(f"scraped/temp_results_{i}.csv", index=False)
            logger.info(f"Průběžné výsledky uloženy do scraped/temp_results_{i}.csv")
    
    # Vytvoření DataFrame z výsledků
    result_df = pd.DataFrame(results, columns=['url', 'xpath_result', 'page_type'])
    
    # Sloučení s původním DataFrame
    final_df = pd.merge(df, result_df, on='url', how='left')
    
    # Uložení výsledků do CSV souboru
    output_path = "scraped/scraped_results.csv"
    final_df.to_csv(output_path, index=False)
    
    # Odstranění dočasných souborů
    for i in range(0, len(urls), batch_size * 10):
        temp_file = f"scraped/temp_results_{i}.csv"
        if os.path.exists(temp_file):
            os.remove(temp_file)
    
    end_time = time.time()
    logger.info(f"Scrapování dokončeno. Výsledky uloženy do {output_path}")
    logger.info(f"Celkový čas: {end_time - start_time:.2f} sekund")

if __name__ == "__main__":
    main()

2025-04-23 12:15:10,825 - INFO - Načítání CSV souboru...
2025-04-23 12:15:11,107 - INFO - Začínám scrapování...
Dávka 10/207: 100%|██████████| 1000/1000 [00:41<00:00, 24.16it/s]
2025-04-23 12:19:42,663 - INFO - Průběžné výsledky uloženy do scraped/temp_results_9000.csv
Dávka 20/207: 100%|██████████| 1000/1000 [02:00<00:00,  8.30it/s]
2025-04-23 12:32:56,679 - INFO - Průběžné výsledky uloženy do scraped/temp_results_19000.csv
Dávka 27/207:  52%|█████▏    | 517/1000 [01:43<01:41,  4.76it/s]

In [ ]:
import pandas as pd
import requests
from lxml import html
import concurrent.futures
import re
import os
import time
from tqdm import tqdm
import logging
import gc
import json
import hashlib

# Nastavení loggeru
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Cesta pro ukládání cache
CACHE_DIR = "script_cache"

def create_cache_key(url):
    """Vytvoří hashovaný klíč z URL"""
    return hashlib.md5(url.encode('utf-8')).hexdigest()

def load_cache(url):
    """Načte cached data pro danou URL"""
    try:
        cache_path = os.path.join(CACHE_DIR, f"{create_cache_key(url)}.json")
        if os.path.exists(cache_path):
            with open(cache_path, 'r', encoding='utf-8') as f:
                cache_data = json.load(f)
                # Kontrola stáří cache (např. max 1 hodina)
                if time.time() - cache_data['timestamp'] < 3600:
                    return cache_data['data']
    except Exception as e:
        logger.warning(f"Chyba při načítání cache: {e}")
    return None

def save_cache(url, data):
    """Uloží data do cache"""
    try:
        # Vytvoření složky pro cache, pokud neexistuje
        os.makedirs(CACHE_DIR, exist_ok=True)
        
        cache_path = os.path.join(CACHE_DIR, f"{create_cache_key(url)}.json")
        cache_data = {
            'timestamp': time.time(),
            'data': data
        }
        
        with open(cache_path, 'w', encoding='utf-8') as f:
            json.dump(cache_data, f)
    except Exception as e:
        logger.warning(f"Chyba při ukládání cache: {e}")

def clean_old_cache(max_age=86400):  # Výchozí max stáří 24h
    """Vyčistí staré cache soubory"""
    try:
        current_time = time.time()
        for filename in os.listdir(CACHE_DIR):
            file_path = os.path.join(CACHE_DIR, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    cache_data = json.load(f)
                    if current_time - cache_data['timestamp'] > max_age:
                        os.remove(file_path)
            except Exception:
                pass
    except Exception as e:
        logger.warning(f"Chyba při čištění cache: {e}")

# Funkce pro scrapování jedné URL adresy s robustnějším error handlingem
def scrape_url(url):
    try:
        # Nejprve zkusíme načíst z cache
        cached_result = load_cache(url)
        if cached_result:
            return cached_result

        # Přidání timeout a exponenciálního backoffu
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        max_retries = 3
        for attempt in range(max_retries):
            try:
                # Přidání malé prodlevy mezi requesty
                time.sleep(0.5)
                
                response = requests.get(url, headers=headers, timeout=10)
                
                if response.status_code != 200:
                    return url, None, None
                
                break
            except requests.exceptions.RequestException as e:
                if attempt == max_retries - 1:
                    logger.error(f"Chyba při opakovaném pokusu o {url}: {str(e)}")
                    return url, None, None
                
                # Exponenciální čekání mezi pokusy
                time.sleep(2 ** attempt)
        
        # Parsování HTML
        tree = html.fromstring(response.content)
        
        # Extrakce dat podle prvního XPath
        xpath_result_1 = tree.xpath("//span[@itemprop='itemListElement']/a/span[@itemprop='name']/text()")
        result_1 = xpath_result_1[0] if xpath_result_1 else None
        
        # Extrakce dat podle druhého XPath (regex)
        page_type = None
        match = re.search(r'"pageType"\s*:\s*"(.*?)"', response.text)
        if match:
            page_type = match.group(1)
        
        # Uložení výsledku do cache
        result = (url, result_1, page_type)
        save_cache(url, result)
        
        return result
    
    except Exception as e:
        logger.error(f"Neočekávaná chyba při scrapování {url}: {str(e)}")
        return url, None, None

# Hlavní funkce
def main():
    start_time = time.time()
    
    # Čištění staré cache před začátkem
    clean_old_cache()
    
    # Vytvoření složky pro výstup, pokud neexistuje
    if not os.path.exists("scraped"):
        os.makedirs("scraped")
    
    # Načtení CSV souboru
    logger.info("Načítání CSV souboru...")
    df = pd.read_csv("50-100_url.csv")
    
    # Kontrola, zda soubor obsahuje sloupec s URL adresami
    if 'url' not in df.columns:
        # Pokud sloupec nemá název 'url', předpokládáme, že první sloupec obsahuje URL adresy
        df.columns = ['url'] + list(df.columns)[1:]
    
    # Získání seznamu URL adres
    urls = df['url'].tolist()
    
    # Inicializace seznamů pro výsledky
    results = []
    
    # Určení optimálního počtu workerů (sníženo pro lepší kontrolu zdrojů)
    max_workers = min(16, os.cpu_count() * 2)
    
    # Zpracování po dávkách pro lepší správu paměti
    batch_size = 500  # Sníženo pro menší zátěž
    total_batches = (len(urls) + batch_size - 1) // batch_size
    
    for i in range(0, len(urls), batch_size):
        # Uvolnění paměti před každou dávkou
        gc.collect()
        
        batch_urls = urls[i:i+batch_size]
        batch_results = []
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Použití tqdm pro zobrazení průběhu
            futures = {executor.submit(scrape_url, url): url for url in batch_urls}
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(batch_urls), desc=f"Dávka {i//batch_size + 1}/{total_batches}"):
                batch_results.append(future.result())
        
        # Přidání výsledků dávky do celkových výsledků
        results.extend(batch_results)
        
        # Průběžné ukládání výsledků (pro případ selhání)
        if (i + batch_size) % (batch_size * 10) == 0 or (i + batch_size) >= len(urls):
            temp_df = pd.DataFrame(results, columns=['url', 'xpath_result', 'page_type'])
            temp_df.to_csv(f"scraped/temp_results_{i}.csv", index=False)
            logger.info(f"Průběžné výsledky uloženy do scraped/temp_results_{i}.csv")
    
    # Vytvoření DataFrame z výsledků
    result_df = pd.DataFrame(results, columns=['url', 'xpath_result', 'page_type'])
    
    # Sloučení s původním DataFrame
    final_df = pd.merge(df, result_df, on='url', how='left')
    
    # Uložení výsledků do CSV souboru
    output_path = "scraped/scraped_results.csv"
    final_df.to_csv(output_path, index=False)
    
    # Odstranění dočasných souborů
    for i in range(0, len(urls), batch_size * 10):
        temp_file = f"scraped/temp_results_{i}.csv"
        if os.path.exists(temp_file):
            os.remove(temp_file)
    
    end_time = time.time()
    logger.info(f"Scrapování dokončeno. Výsledky uloženy do {output_path}")
    logger.info(f"Celkový čas: {end_time - start_time:.2f} sekund")

if __name__ == "__main__":
    main()


2025-04-23 10:16:56,880 - INFO - Načítání CSV souboru...
Dávka 10/413: 100%|██████████| 500/500 [00:24<00:00, 20.27it/s]
2025-04-23 10:19:27,762 - INFO - Průběžné výsledky uloženy do scraped/temp_results_4500.csv
Dávka 20/413: 100%|██████████| 500/500 [00:25<00:00, 19.30it/s]
2025-04-23 10:23:45,216 - INFO - Průběžné výsledky uloženy do scraped/temp_results_9500.csv
Dávka 30/413: 100%|██████████| 500/500 [00:33<00:00, 14.96it/s]
2025-04-23 10:28:45,866 - INFO - Průběžné výsledky uloženy do scraped/temp_results_14500.csv
Dávka 40/413: 100%|██████████| 500/500 [01:04<00:00,  7.71it/s]
2025-04-23 10:36:46,393 - INFO - Průběžné výsledky uloženy do scraped/temp_results_19500.csv
Dávka 50/413: 100%|██████████| 500/500 [01:43<00:00,  4.83it/s]
2025-04-23 10:50:47,924 - INFO - Průběžné výsledky uloženy do scraped/temp_results_24500.csv
Dávka 51/413:  93%|█████████▎| 464/500 [01:41<00:07,  5.07it/s]